In [7]:
!pip install gmaps

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key


In [9]:
#read in the output from Part1
data_df = pd.read_csv('../WeatherPy/output_data.csv')
#drop a column
#deleting the strange column
data_df1 = data_df.drop(["Unnamed: 0"], axis = 1)
data_df1

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chokurdakh,70.6333,147.9167,43.27,72,96,12.46,RU,1627344000
1,Kamsack,51.5667,-101.9009,84.88,23,16,7.90,CA,1627344000
2,Lorengau,-2.0226,147.2712,84.88,74,72,3.40,PG,1627344000
3,Longyearbyen,78.2186,15.6401,43.18,87,83,0.94,SJ,1627344000
4,Hindupur,13.8289,77.4933,72.12,82,93,9.42,IN,1627344000
...,...,...,...,...,...,...,...,...,...
578,Tabou,4.4230,-7.3528,76.39,76,100,9.73,CI,1627344000
579,Mattru,7.6244,-11.8332,73.53,96,64,3.22,SL,1627344000
580,Oranjemund,-28.5500,16.4333,62.55,57,0,6.62,NaN,1627344000
581,Nichinan,31.6000,131.3667,81.50,82,10,0.94,JP,1627344000


In [10]:
#configure gmaps
gmaps.configure(api_key=g_key)
#Create a variable that stores coordinates
coord = data_df1[['Latitude','Longitude']]

#create a seperate dataframe for with humidity
humidity = data_df1['Humidity'].astype(float)

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

#create heat layer based on humidity
heat_layer = gmaps.heatmap_layer(coord, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1, opacity=1)

#add layer to the heatmap
fig.add_layer(heat_layer)
plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [11]:
# **Create new DataFrame fitting weather criteria

#**Narrow down the cities to fit weather conditions.Drop any rows will null values.

vacay_spot = data_df.loc[(data_df['Max Temp'] <=98) &
                         (data_df['Max Temp'] >=75) & 
                         (data_df['Humidity'] <=90) &
                         (data_df['Humidity'] <=20), :].dropna()
vacay_spot

#Observation
#Based on our search criteria, we identified 10 cities that fit our criteria for a possible
#vacation location 

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,Winnemucca,40.9730,-117.7357,96.84,17,35,14.52,US,1627344000
66,66,Dŭstí,37.3481,68.6640,85.95,18,1,5.99,TJ,1627344000
216,216,Nālūt,30.3333,10.8500,85.69,18,0,15.64,LY,1627344000
259,259,Adrar,20.5022,-10.0711,95.16,12,1,13.80,MR,1627344000
273,273,El Dorado,38.7666,-120.5177,87.93,15,92,8.52,US,1627344000
353,353,Laramie,41.3114,-105.5911,83.82,17,32,15.77,US,1627344000
401,401,Chaman,30.9236,66.4512,85.21,12,27,6.42,PK,1627344000
433,433,Gazojak,41.1875,61.4036,83.44,18,0,11.79,TM,1627344000
450,450,Atar,20.5169,-13.0499,97.00,17,26,3.40,MR,1627344000
506,506,Buraidah,26.3260,43.9750,97.12,15,32,12.06,SA,1627344000


In [16]:
hotel_df = vacay_spot

hotel_df['Hotel Name'] = np.nan

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping to next city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping to next city.
Hotel not found...skipping to next city.
Hotel not found...skipping to next city.


In [17]:
#Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
48,48,Winnemucca,40.9730,-117.7357,96.84,17,35,14.52,US,1627344000,Winners Inn Casino
66,66,Dŭstí,37.3481,68.6640,85.95,18,1,5.99,TJ,1627344000,Hotel Izzat
273,273,El Dorado,38.7666,-120.5177,87.93,15,92,8.52,US,1627344000,South Fork Group Campground
401,401,Chaman,30.9236,66.4512,85.21,12,27,6.42,PK,1627344000,Niamat parda Center chaman
433,433,Gazojak,41.1875,61.4036,83.44,18,0,11.79,TM,1627344000,In gowy adamlaryñ yaśayan oyi
450,450,Atar,20.5169,-13.0499,97.00,17,26,3.40,MR,1627344000,Oumou Elghoura
506,506,Buraidah,26.3260,43.9750,97.12,15,32,12.06,SA,1627344000,فندق راحة السلام


In [18]:
#Observations
#our analysis revealed that out of the 10 destinations, there were a total of 7 hotels that was in a 5000m
#radius of our ideal cities for a vacation.  The names of the hotels were included above. 


# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_df.iterrows()]
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [23]:

# Add a marker layer for each city to the map. 
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

AssertionError: 